In [1]:
import sympy as sp
from typing import List
from pyomo.core.expr.sympy_tools import sympy2pyomo_expression
import pyomo.environ as pyo
from sympy import symbols, exp
from pyomo.environ import *

In [2]:
def build_stochastic_flex_piecewise(
    design_vars: List[sp.Symbol],
    theta_expr: sp.Matrix,
    region_constraints: List[tuple],
    flex_expressions: List[sp.Expr]
) -> sp.Piecewise:
    """
    Build a symbolic Piecewise stochastic flexibility expression.

    Parameters:
        design_vars: list of sympy symbols [d1, d2, ...]
        theta_expr: sympy Matrix of theta as a function of design vars
        region_constraints: list of (E, f) tuples where E @ theta <= f
        flex_expressions: symbolic expressions (one per region)

    Returns:
        sympy Piecewise expression of stochastic flexibility
    """
    assert len(region_constraints) == len(flex_expressions), "Region count must match expression count"

    piecewise_cases = []

    for (E, f), flex_expr in zip(region_constraints, flex_expressions):
        E = sp.Matrix(E)
        f = sp.Matrix(f)
        ineqs = E @ theta_expr - f
        region_condition = sp.And(*[ineq <= 0 for ineq in ineqs])
        piecewise_cases.append((flex_expr, region_condition))

    # Add default case
    piecewise_cases.append((0, True))

    return sp.Piecewise(*piecewise_cases)


In [3]:
# Design variables
d1, d2 = sp.symbols('d1 d2')

# Theta = A*d + b
theta_expr = sp.Matrix([
    2*d1 - d2 + 1,
    d1 + d2
])

# Region 1
E1 = [[1, -1], [-1, -1]]
f1 = [2, -4]
flex1 = (d1 + d2)**2

# Region 2
E2 = [[0, 1], [1, 0]]
f2 = [1, 3]
flex2 = d1 * d2 + 1

# Call the function
piecewise_flex = build_stochastic_flex_piecewise(
    design_vars=[d1, d2],
    theta_expr=theta_expr,
    region_constraints=[(E1, f1), (E2, f2)],
    flex_expressions=[flex1, flex2]
)

# Output the expression
sp.pprint(piecewise_flex, use_unicode=True)


⎧         2                                 
⎪(d₁ + d₂)     for d₁ ≥ 1 ∧ d₁ - 2⋅d₂ ≤ 1   
⎪                                           
⎨d₁⋅d₂ + 1   for d₁ + d₂ ≤ 1 ∧ 2⋅d₁ - d₂ ≤ 2
⎪                                           
⎪    0                  otherwise           
⎩                                           


In [4]:
# def create_pyomo_model_from_stocflex_expr(sym_expr, design_var_names, target_value=0.5):
#     model = pyo.ConcreteModel()
# 
#     if design_var_names:
#         model.design_vars = pyo.Var(design_var_names, initialize=1.0, domain=pyo.Reals)
# 
#         # Create symbol mapping
#         sym_vars = sp.symbols(design_var_names)
#         sym_map = {s: model.design_vars[str(s)] for s in sym_vars}
# 
#         # Substitute Pyomo vars into SymPy expression
#         substituted_expr = sym_expr.subs(sym_map)
# 
#         # Convert to Pyomo-native expression tree
#         pyomo_expr = sympy2pyomo_expression(substituted_expr)
# 
#         model.stocflex_expr = pyo.Expression(expr=pyomo_expr)
#     else:
#         # Handle constant case (no design variables)
#         model.stocflex_expr = pyo.Expression(expr=float(sym_expr.evalf()))
# 
#     model.flexibility_constraint = pyo.Constraint(expr=model.stocflex_expr >= target_value)
#     return model

In [5]:
# # Suppose this is your SymPy expression
# d1, d2 = symbols("d1 d2")
# expr = (0.1*d1 + 0.2*d2 + 0.4) * sp.exp(-((0.3*d1 + 0.3*d2 + 0.1) - 1)**2)
# 
# # Create Pyomo model
# m = create_pyomo_model_from_stocflex_expr(expr, design_var_names=["d1", "d2"], target_value=0.6)
# 
# # Now model is ready to be solved
# solver = pyo.SolverFactory("ipopt")
# solver.solve(m)

In [9]:
# Define symbolic expression
d1, d2 = sp.symbols("d1 d2")
sym_expr = (0.1*d1 + 0.2*d2 + 0.4) * sp.exp(-((0.3*d1 + 0.3*d2 + 0.1) - 1)**2)

# Create Pyomo model
model = pyo.ConcreteModel()
model.design_vars = pyo.Var(['d1', 'd2'], domain=pyo.Reals, initialize=1.0)

# Step 1: SymPy symbol to Pyomo variable mapping
sym_map = {d1: model.design_vars['d1'], d2: model.design_vars['d2']}

# # Step 2: Substitute and convert
# substituted = sym_expr.subs(sym_map)
# pyomo_expr = sympy2pyomo_expression(substituted)

# Step 3: Use in Pyomo expression
# model.stocflex_expr = pyo.Expression(expr=pyomo_expr)
# model.flexibility_constraint = pyo.Constraint(expr=model.stocflex_expr >= 0.5)

In [7]:
help(sympy2pyomo_expression)

Help on function sympy2pyomo_expression in module pyomo.core.expr.sympy_tools:

sympy2pyomo_expression(expr, object_map)


In [10]:
substituted = sym_expr.subs(sym_map)

TypeError: Implicit conversion of Pyomo numeric value (design_vars[d1]) to float is disabled.
This error is often the result of using Pyomo components as arguments to
one of the Python built-in math module functions when defining
expressions. Avoid this error by using Pyomo-provided math functions or
explicitly resolving the numeric value using the Pyomo value() function.

In [11]:
help(sym_expr)

Help on Mul in module sympy.core.mul object:

class Mul(sympy.core.expr.Expr, sympy.core.operations.AssocOp)
 |  Mul(*args, evaluate=None, _sympify=True)
 |  
 |  Expression representing multiplication operation for algebraic field.
 |  
 |  .. deprecated:: 1.7
 |  
 |     Using arguments that aren't subclasses of :class:`~.Expr` in core
 |     operators (:class:`~.Mul`, :class:`~.Add`, and :class:`~.Pow`) is
 |     deprecated. See :ref:`non-expr-args-deprecated` for details.
 |  
 |  Every argument of ``Mul()`` must be ``Expr``. Infix operator ``*``
 |  on most scalar objects in SymPy calls this class.
 |  
 |  Another use of ``Mul()`` is to represent the structure of abstract
 |  multiplication so that its arguments can be substituted to return
 |  different class. Refer to examples section for this.
 |  
 |  ``Mul()`` evaluates the argument unless ``evaluate=False`` is passed.
 |  The evaluation logic includes:
 |  
 |  1. Flattening
 |      ``Mul(x, Mul(y, z))`` -> ``Mul(x, y, z)``